# Tarık Emre Talay - 118200077

In [106]:
!pip install contractions

     -------------------------------------- 289.9/289.9 kB 2.2 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import re
import string
import contractions

### Reading File

In [2]:
with open("amazon_review_random.txt") as f:
    contents = f.readlines()

In [3]:
len(contents)

29

In [4]:
df = pd.DataFrame(contents, columns= ["text"])
df

,text
0,"Purchased this for my device, it worked as adv..."
1,it works as expected. I should have sprung for...
2,This think has worked out great.Had a diff. br...
3,I've had this for a few months and have had ZE...
4,"Bought it with Retail Packaging, arrived legit..."
5,It's mini storage. It doesn't do anything els...
6,It does what it's supposed to and SanDisk has ...
7,I have it in my phone and it never skips a bea...
8,It's hard to believe how affordable digital ha...
9,Works in a HTC Rezound. Was running short of ...


### Data Cleaning
#### Placed into another column as cleaned_text

In [5]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_wordss = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC_3876_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def clean_data(x):
    x = contractions.fix(x) # Expanding Contractions
    x = x.lower() # Making all lowercase
    x =  re.sub('[%s]' % re.escape(string.punctuation), '' , x) # Removing punctuations
    x = re.sub(r'\d+', '', x) # Removing numbers    
    x = " ".join([w for w in x.split() if not w.lower() in stop_wordss]) # Removing stopwords
    x =(" ".join(re.findall("[a-zA-Z,.]+",x))) # For only words remain
    x = " ".join(x.split())
    return x

In [7]:
df["cleaned_text"] = df["text"].apply(lambda x: clean_data(x))

df

### Spelling Correction by using model from huggingface

In [9]:
from transformers import pipeline
fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

C:\Users\PC_3876_\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC_3876_\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
from transformers import pipeline
fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

In [12]:
df["cleaned_text"] = df["cleaned_text"].apply(lambda x:fix_spelling(x,max_length=2048))

In [1]:
# Converted cleaned spelled text from dict to string
df["cleaned_text"] = df["cleaned_text"].apply(lambda x: x[0]["generated_text"])

NameError: name 'df' is not defined

In [24]:
df["cleaned_text"]

0     Purchased device worked advertised never much ...
1     Works expected sprung higher capacity think ma...
2     I think worked great had different brand gb ca...
3     months zero issues since transferred note note...
4     Bought retail packaging arrived legit orange e...
5     Mini storage anything else supposed purchased,...
6     Supposed Sandisk's long standing reputation sp...
7     Phone never skips beat file transfers, speedy ...
8     Hard believe affordable digital become gb devi...
9     Works that rezound running short space go sand...
10    Galaxy super fast card totally happy happy sti...
11    Like sd card take music video downloads, perso...
12    Works file writes bit slower expected usb read...
13                           Name speaks go sandisk go.
14    Solid HD card fast reading, writing fast recor...
15    heard cards write speed insufficient, however,...
16    Bought use go pro hero black edition requires ...
17    Got gig one filled kept getting insufficie

### Take into texts from data frame to string

In [27]:
text_last = " ".join(df.cleaned_text)

In [28]:
text_last

"Purchased device worked advertised never much phone memory since download lot stuff brainer. Works expected sprung higher capacity think made bit cheesier earlier versions paint looks clean. I think worked great had different brand gb card went south months this one held pretty well since note. months zero issues since transferred note note card reliable solidcheers. Bought retail packaging arrived legit orange envelope English version Asian like picture shows arrived quickly bought retail packaging, htc one sv lg optimus cards working order probably best price get nice sd card. Mini storage anything else supposed purchased, add additional storage microsoft surface pro tablet come gb. Supposed Sandisk's long standing reputation speaks. Phone never skips beat file transfers, speedy corruption issues, memory fade issues would expect Sandisk brand great card entrust precious files slightly cheaper piece crap lose everything for spending extra couple bucks trusted product goes good data. 

### Byte Pair Encoding Part

In [29]:
## get the word frequency and add the end of word (</w>) token at the end of each word
words = text_last.strip().split(" ")
print(f"Vocabulary size: {len(words)}")

Vocabulary size: 628


In [30]:
import collections
word_freq_dict = collections.defaultdict(int)

for word in words:
    word_freq_dict[' '.join(word) + ' </w>'] += 1

word_freq_dict

defaultdict(int,
            {'P u r c h a s e d </w>': 1,
             'd e v i c e </w>': 2,
             'w o r k e d </w>': 2,
             'a d v e r t i s e d </w>': 1,
             'n e v e r </w>': 3,
             'm u c h </w>': 1,
             'p h o n e </w>': 5,
             'm e m o r y </w>': 2,
             's i n c e </w>': 5,
             'd o w n l o a d </w>': 1,
             'l o t </w>': 1,
             's t u f f </w>': 1,
             'b r a i n e r . </w>': 1,
             'W o r k s </w>': 6,
             'e x p e c t e d </w>': 3,
             's p r u n g </w>': 1,
             'h i g h e r </w>': 1,
             'c a p a c i t y </w>': 1,
             't h i n k </w>': 3,
             'm a d e </w>': 1,
             'b i t </w>': 2,
             'c h e e s i e r </w>': 1,
             'e a r l i e r </w>': 1,
             'v e r s i o n s </w>': 1,
             'p a i n t </w>': 1,
             'l o o k s </w>': 1,
             'c l e a n . </w>': 1,
       

In [31]:
char_freq_dict = collections.defaultdict(int)
for word, freq in word_freq_dict.items():
    chars = word.split()
    for char in chars:
        char_freq_dict[char] += freq

char_freq_dict

defaultdict(int,
            {'P': 2,
             'u': 95,
             'r': 258,
             'c': 147,
             'h': 85,
             'a': 282,
             's': 287,
             'e': 420,
             'd': 196,
             '</w>': 628,
             'v': 37,
             'i': 226,
             'w': 44,
             'o': 206,
             'k': 60,
             't': 226,
             'n': 194,
             'm': 69,
             'p': 114,
             'y': 53,
             'l': 154,
             'f': 62,
             'b': 48,
             '.': 32,
             'W': 6,
             'x': 9,
             'g': 100,
             'I': 3,
             'z': 5,
             'B': 2,
             'E': 1,
             'A': 6,
             'q': 10,
             ',': 33,
             'M': 2,
             'S': 10,
             "'": 1,
             'H': 3,
             'G': 5,
             'L': 1,
             'N': 2,
             'D': 3,
             'C': 1,
             'T': 1,
             'R

In [32]:
## create all possible consecutive pairs
pairs = collections.defaultdict(int)
for word, freq in word_freq_dict.items():
    chars = word.split()
    for i in range(len(chars)-1):
        pairs[chars[i], chars[i+1]] += freq

pairs

defaultdict(int,
            {('P', 'u'): 1,
             ('u', 'r'): 12,
             ('r', 'c'): 6,
             ('c', 'h'): 8,
             ('h', 'a'): 10,
             ('a', 's'): 24,
             ('s', 'e'): 20,
             ('e', 'd'): 47,
             ('d', '</w>'): 97,
             ('d', 'e'): 25,
             ('e', 'v'): 13,
             ('v', 'i'): 11,
             ('i', 'c'): 29,
             ('c', 'e'): 30,
             ('e', '</w>'): 97,
             ('w', 'o'): 16,
             ('o', 'r'): 40,
             ('r', 'k'): 16,
             ('k', 'e'): 12,
             ('a', 'd'): 24,
             ('d', 'v'): 2,
             ('v', 'e'): 22,
             ('e', 'r'): 56,
             ('r', 't'): 7,
             ('t', 'i'): 21,
             ('i', 's'): 28,
             ('n', 'e'): 29,
             ('r', '</w>'): 30,
             ('m', 'u'): 5,
             ('u', 'c'): 9,
             ('h', '</w>'): 9,
             ('p', 'h'): 7,
             ('h', 'o'): 13,
             ('o', 'n')

In [41]:
word_freq_dict = collections.defaultdict(int)

for word in words:
    word_freq_dict[' '.join(word) + ' </w>'] += 1

##find the best pair

def get_pairs(word_freq_dict):
    pairs = collections.defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for i in range(len(chars)-1):
            pairs[chars[i], chars[i+1]] += freq
    return pairs

def merge_byte_pairs(best_pair, word_freq_dict):
    print(best_pair)
    merged_dict = {}
    bigram = re.escape(' '.join(best_pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in word_freq_dict:
        # print(word)
        w_out = p.sub(''.join(best_pair), word)
        merged_dict[w_out] = word_freq_dict[word]
    return merged_dict

def get_subword_tokens(word_freq_dict):
    char_freq_dict = collections.defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for char in chars:
            char_freq_dict[char] += freq
    return char_freq_dict

for i in range(2000):
    pairs = get_pairs(word_freq_dict)
    best_pair = max(pairs, key=pairs.get)
    print(f"Iteration {i}: ")
    word_freq_dict = merge_byte_pairs(best_pair, word_freq_dict)
    # print(word_freq_dict)
    subword_tokens = get_subword_tokens(word_freq_dict)
    print(subword_tokens)
    print(len(subword_tokens))
    print("--------")

 
word_freq_dict

Iteration 0: 
('d', '</w>')
defaultdict(<class 'int'>, {'P': 2, 'u': 95, 'r': 258, 'c': 147, 'h': 85, 'a': 282, 's': 287, 'e': 420, 'd</w>': 97, 'd': 99, 'v': 37, 'i': 226, '</w>': 531, 'w': 44, 'o': 206, 'k': 60, 't': 226, 'n': 194, 'm': 69, 'p': 114, 'y': 53, 'l': 154, 'f': 62, 'b': 48, '.': 32, 'W': 6, 'x': 9, 'g': 100, 'I': 3, 'z': 5, 'B': 2, 'E': 1, 'A': 6, 'q': 10, ',': 33, 'M': 2, 'S': 10, "'": 1, 'H': 3, 'G': 5, 'L': 1, 'N': 2, 'D': 3, 'C': 1, 'T': 1, 'R': 2, 'Q': 1, '’': 2, '-': 1})
49
--------
Iteration 1: 
('e', '</w>')
defaultdict(<class 'int'>, {'P': 2, 'u': 95, 'r': 258, 'c': 147, 'h': 85, 'a': 282, 's': 287, 'e': 323, 'd</w>': 97, 'd': 99, 'v': 37, 'i': 226, 'e</w>': 97, 'w': 44, 'o': 206, 'k': 60, 't': 226, 'n': 194, '</w>': 434, 'm': 69, 'p': 114, 'y': 53, 'l': 154, 'f': 62, 'b': 48, '.': 32, 'W': 6, 'x': 9, 'g': 100, 'I': 3, 'z': 5, 'B': 2, 'E': 1, 'A': 6, 'q': 10, ',': 33, 'M': 2, 'S': 10, "'": 1, 'H': 3, 'G': 5, 'L': 1, 'N': 2, 'D': 3, 'C': 1, 'T': 1, 'R': 2, 'Q': 1

defaultdict(<class 'int'>, {'P': 2, 'urchas': 3, 'ed</w>': 23, 'de': 4, 'v': 12, 'ice</w>': 4, 'work': 4, 'ad': 10, 'ver': 4, 't': 40, 'is': 5, 'never</w>': 3, 'm': 33, 'uc': 2, 'h</w>': 3, 'phone</w>': 5, 'e': 35, 'or': 18, 'y</w>': 19, 'since</w>': 5, 'd': 26, 'ow': 4, 'n': 16, 'lo': 9, 'ad</w>': 4, 't</w>': 32, 'st': 18, 'u': 23, 'f': 17, 'f</w>': 2, 'b': 21, 'r': 16, 'ain': 5, 'er': 21, '.</w>': 19, 'Works</w>': 6, 'expected</w>': 3, 'sp': 5, 'ru': 4, 'ng</w>': 4, 'high': 3, 'er</w>': 16, 'c': 31, 'ap': 4, 'ac': 5, 'ity</w>': 4, 'think</w>': 3, 'e</w>': 39, 'it</w>': 4, 'ch': 4, 'ee': 5, 'si': 7, 'ar': 13, 'li': 10, 'on': 15, 's</w>': 40, 'p': 31, 'o': 30, 'k': 16, 'le': 7, 'an': 13, 'I': 3, '</w>': 31, 'great</w>': 8, 'h': 21, 'di': 6, 'ff': 6, 'en': 21, 'brand</w>': 5, 'gb': 4, 'card</w>': 16, 'w': 20, 's': 45, 'ou': 9, 'th</w>': 4, 'th': 11, 'i': 32, 'one</w>': 8, 'el': 8, 'd</w>': 14, 're': 25, 'ell</w>': 4, 'no': 8, 'e.</w>': 8, 'z': 5, 'o</w>': 5, 'issues</w>': 5, 'transf': 4

defaultdict(<class 'int'>, {'P': 2, 'urchas': 1, 'ed</w>': 15, 'device</w>': 2, 'worked</w>': 2, 'ad': 8, 'ver': 4, 't': 17, 'is': 5, 'never</w>': 3, 'm': 11, 'uc': 2, 'h</w>': 3, 'phone</w>': 5, 'memory</w>': 2, 'since</w>': 5, 'downlo': 2, 'ad</w>': 4, 'lo': 4, 't</w>': 17, 'stu': 2, 'f': 12, 'f</w>': 2, 'b': 8, 'r': 7, 'ain': 3, 'er.</w>': 2, 'Works</w>': 6, 'expected</w>': 3, 'sp': 3, 'ru': 2, 'ng</w>': 1, 'high': 3, 'er</w>': 8, 'capac': 2, 'ity</w>': 1, 'think</w>': 3, 'mad': 2, 'e</w>': 17, 'bit</w>': 2, 'ch': 2, 'ee': 5, 'si': 5, 'ear': 2, 'li': 6, 'on': 4, 's</w>': 18, 'p': 15, 'o': 14, 'ks</w>': 3, 'c': 12, 'le': 5, 'an': 6, '.</w>': 14, 'I': 3, '</w>': 12, 'great</w>': 8, 'h': 7, 'differ': 2, 'ent</w>': 3, 'brand</w>': 5, 'gb</w>': 2, 'card</w>': 16, 'w': 9, 'sou': 2, 'th</w>': 4, 'months</w>': 2, 'th': 3, 'i': 18, 'one</w>': 8, 'hel': 2, 'd</w>': 5, 'pre': 3, 'tt': 3, 'y</w>': 7, 'well</w>': 3, 'not': 1, 'e.</w>': 6, 'z': 3, 'ero</w>': 2, 'issues</w>': 5, 'transfer': 2, 're

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [42]:
for key,value in subword_tokens.items():
  print(f'key:{key} - value:{value}')

key:Purchased</w> - value:1
key:device</w> - value:2
key:worked</w> - value:2
key:advertised</w> - value:1
key:never</w> - value:3
key:much</w> - value:1
key:phone</w> - value:5
key:memory</w> - value:2
key:since</w> - value:5
key:download</w> - value:1
key:lot</w> - value:1
key:stuff</w> - value:1
key:brainer.</w> - value:1
key:Works</w> - value:6
key:expected</w> - value:3
key:sprung</w> - value:1
key:higher</w> - value:1
key:capacity</w> - value:1
key:think</w> - value:3
key:made</w> - value:1
key:bit</w> - value:2
key:cheesier</w> - value:1
key:earlier</w> - value:1
key:versions</w> - value:1
key:paint</w> - value:1
key:looks</w> - value:1
key:clean.</w> - value:1
key:I</w> - value:1
key:great</w> - value:8
key:had</w> - value:1
key:different</w> - value:1
key:brand</w> - value:5
key:gb</w> - value:2
key:card</w> - value:16
key:went</w> - value:1
key:south</w> - value:1
key:months</w> - value:2
key:this</w> - value:1
key:one</w> - value:7
key:held</w> - value:1
key:pretty</w> - val